<h1 style="font-size: 30px"> DESAFIO SANTANDER DEV WEEK 2023 </h1>

# 1. Introdução

O código deste Notebook realiza a extração de dados de um banco de dados criado e aplica um processo de ETL (Extração, Transformação e Carregamento) com o objetivo de gerar valor a partir das informações dos usuários. A finalidade principal é fornecer mensagens personalizadas de investimento, oferecendo insights financeiros relevantes e estratégias de economia adaptadas à situação de cada usuário.

# 2. Preparação dos Dados

O processo de preparação dos dados envolve a criação de um banco de dados e a inserção de dados fictícios nas tabelas correspondentes, representando usuários, contas e cartões. Essa etapa simula um ambiente realista, permitindo a execução das operações de Extração, Transformação e Carregamento (ETL) em dados semelhantes aos reais. Isso viabiliza a geração de mensagens personalizadas de investimento com base em métricas financeiras calculadas a partir dos dados inseridos, oferecendo insights valiosos aos usuários.

## 2.1. SQLite

O Sistema de Gerenciamento de Banco de Dados (SGBD) escolhido foi o SQLite devido à sua leveza e versatilidade para armazenar, gerenciar e recuperar dados. Sua natureza autônoma elimina a necessidade de um servidor externo. Com suporte a consultas SQL, índices e transações, o SQLite atende às demandas do desafio, oferecendo eficiência no gerenciamento de dados locais.

In [1]:
import sqlite3

## 2.2. Criação das Tabelas

O processo de criação das tabelas no banco de dados segue um fluxo bem definido. Primeiramente, são elaboradas instruções SQL que especificam a estrutura das tabelas, então é estabelecida uma conexão com o banco de dados. Utilizando essa conexão, as tabelas são criadas no banco de dados por meio da execução das instruções.

### 2.2.1. Tabela: Users

A tabela `users` armazena informações básicas sobre os usuários, como um identificador único (id) e o nome do usuário. Esta tabela é a base de referência para as outras tabelas, permitindo a associação de informações individuais com cada usuário.

In [2]:
create_table_users = '''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(50)
    );
'''

### 2.2.2. Tabela: Accounts

A tabela `accounts` contém detalhes das contas financeiras dos usuários. Isso inclui informações como o número da conta, a agência bancária, o saldo da conta e o ID do usuário ao qual a conta pertence. A coluna `user_id` estabelece uma relação entre a conta e o usuário correspondente na tabela `users`.

In [3]:
create_table_accounts = '''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        number VARCHAR(6) UNIQUE,
        agency VARCHAR(4),
        balance REAL,
        user_id INTEGER,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
'''

### 2.2.3. Tabela: Cards

A tabela `cards` armazena dados relacionados aos cartões de crédito dos usuários. Isso inclui o número do cartão, a utilização atual do cartão, o limite do cartão e o ID do usuário associado. A coluna `account_id` estabelece uma relação entre o cartão e a conta correspondente na tabela `accounts`.

In [4]:
create_table_cards = '''
    CREATE TABLE cards (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        number VARCHAR(16) UNIQUE,
        usage REAL,
        card_limit REAL,
        account_id INTEGER,
        FOREIGN KEY (account_id) REFERENCES accounts(id)
    );
'''

### 2.2.4. Tabela: News

A tabela `news` armazena mensagens que serão entregues aos usuários. Cada registro na tabela inclui um ID exclusivo, o ID do usuário associado à mensagem e o conteúdo da mensagem em si.

In [5]:
create_table_news = '''
    CREATE TABLE news (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        message TEXT,
        user_id INTEGER,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
'''

### 2.2.5. Criando as Tabelas no Banco

Uma vez que as instruções SQL estejam definidas, é realizada a conexão com o banco de dados `SDW2023.db` e a criação das tabelas, seguindo as instruções predefinidas.

In [6]:
conn = sqlite3.connect('SDW2023.db')
cur = conn.cursor()
cur.execute(create_table_users)
cur.execute(create_table_accounts)
cur.execute(create_table_cards)
cur.execute(create_table_news)
conn.commit()
cur.close()
conn.close()

## 2.3. Inserção dos Dados

### 2.3.1. Dados

Nesta etapa, são definidos dados fictícios que simulam um ambiente real. Os dados são estruturados em listas de tuplas, onde cada lista representa uma tabela e cada tupla simula um registro correspondente. Cada tupla inclui informações pertinentes às entidades previamente definidas, como nomes de usuários, números de contas e números de cartões.

É importante ressaltar que, por enquanto, nenhuma informação será atribuída à tabela `news`. Os dados a serem inseridos nessa tabela serão tratados durante a fase de transformação dos dados.

A definição destes dados cobrem todas as variedades possíveis dentro deste contexto bancário:
- Usuário com uma conta e nenhum cartão;
- Usuário com uma conta e um ou dois cartões;
- Usuário com duas contas e nenhum, um ou dois cartões para cada conta.

In [7]:
users = [
    ('Maria Alice Santos',),
    ('Ronaldo Nunes',),
    ('Paulo Moreira',),
    ('Ana Rodrigues',),
    ('Carlos Lima',),
    ('Lúcia Almeida',),
    ('Fernando Sousa',),
    ('Mariana Costa',),
    ('José Silva',),
]

accounts = [
    ('000001', '0001', 100.0, 1),
    ('000002', '0002', 200.0, 2),
    ('000003', '0003', 300.0, 3),
    ('000004', '0003', 400.0, 3),
    ('000005', '0001', 500.0, 4),
    ('000006', '0001', 600.0, 4),
    ('000007', '0002', 700.0, 5),
    ('000008', '0002', 800.0, 5),
    ('000009', '0003', 900.0, 6),
    ('000010', '0004', 1000.0, 7),
    ('000011', '0004', 1100.0, 7),
    ('000012', '0002', 1200.0, 8),
    ('000013', '0002', 1300.0, 8),
    ('000014', '0001', 1400.0, 9),
    ('000015', '0001', 1500.0, 9),
]

cards = [
    ('**** **** **** 0001', 1000.0, 1000.0, 2),
    ('**** **** **** 0002', 1000.0, 1500.0, 5),
    ('**** **** **** 0003', 1000.0, 2000.0, 7),
    ('**** **** **** 0004', 1000.0, 2500.0, 8),
    ('**** **** **** 0005', 1000.0, 3000.0, 9),
    ('**** **** **** 0006', 500.0, 500.0, 9),
    ('**** **** **** 0007', 500.0, 1000.0, 10),
    ('**** **** **** 0008', 500.0, 1500.0, 10),
    ('**** **** **** 0009', 500.0, 2000.0, 11),
    ('**** **** **** 0010', 500.0, 2500.0, 11),
    ('**** **** **** 0011', 500.0, 3000.0, 12),
    ('**** **** **** 0012', 250.0, 250.0, 12),
    ('**** **** **** 0013', 250.0, 500.0, 13),
    ('**** **** **** 0014', 250.0, 750.0, 15),
    ('**** **** **** 0015', 250.0, 1000.0, 15),
]

### 2.3.2. Inserção dos Dados no Banco

Com os dados estruturados, é feita uma nova conexão com o banco. Utilizando um cursor, ele itera pelas listas de usuarios, contas e cartões, executando instruções SQL para inserir os dados definidos nas tabelas correspondentes, finalizando a etapa de preparação dos dados.

In [8]:
conn = sqlite3.connect('SDW2023.db')
cur = conn.cursor()

for user in users:
    cur.execute('INSERT INTO users (name) VALUES (?)', user)

for account in accounts:
    cur.execute('INSERT INTO accounts (number, agency, balance, user_id) VALUES (?, ?, ?, ?)', account)

for card in cards:
    cur.execute('INSERT INTO cards (number, usage, card_limit, account_id) VALUES (?, ?, ?, ?)', card)

conn.commit()
conn.close()

# 3. Extração, Transformação e Carregamento

O processo de ETL neste desafio é iniciado com a extração de dados das tabelas SQL. Na etapa subsequente, os dados passam por uma transformação para agregar valor aos dados através de mensagens com estratégias de economia. Por fim, as mensagens personalizadas são carregadas na tabela news, que podem ser consumidas por um sistema de envio de mensagens aos usuários.

Para realizar esse processo, será utilizada a biblioteca `pandas`, que possibilita a manipulação e análise eficiente dos dados.

In [9]:
import pandas as pd

## 3.1. Extração

Nessa etapa, os dados são extraídos das tabelas SQL por meio de uma consulta SQL elaborada para unir informações relevantes de diferentes tabelas. A biblioteca `pandas` é usada para transformar os resultados em um `DataFrame`, permitindo a análise e manipulação eficaz dos dados. O processo resulta na visualização tabular dos dados extraídos, fornecendo a base para as etapas seguintes de transformação.

In [10]:
query = '''
    SELECT u.id AS user_id, u.name AS username, c.id AS card_id, c.number AS card_number, c.usage AS card_usage, c.card_limit
    FROM users u
    JOIN accounts a ON u.id = a.user_id
    FULL JOIN cards c ON a.id = c.account_id
'''

conn = sqlite3.connect('SDW2023.db')
df = pd.read_sql(query, conn)
conn.close()

display(df)

,user_id,username,card_id,card_number,card_usage,card_limit
0,1,Maria Alice Santos,NaN,None,NaN,NaN
1,2,Ronaldo Nunes,1.0,**** **** **** 0001,1000.0,1000.0
2,3,Paulo Moreira,NaN,None,NaN,NaN
3,3,Paulo Moreira,NaN,None,NaN,NaN
4,4,Ana Rodrigues,2.0,**** **** **** 0002,1000.0,1500.0
5,4,Ana Rodrigues,NaN,None,NaN,NaN
6,5,Carlos Lima,3.0,**** **** **** 0003,1000.0,2000.0
7,5,Carlos Lima,4.0,**** **** **** 0004,1000.0,2500.0
8,6,Lúcia Almeida,5.0,**** **** **** 0005,1000.0,3000.0
9,6,Lúcia Almeida,6.0,**** **** **** 0006,500.0,500.0


# 3.2.Transformação

Para a transformação, é gerado valor através da verificação da porcentagem de uso do cartão de crédito do usuário, com o propósito de atribuir uma mensagem de alerta personalizada com base em seu padrão de uso.

### 3.2.1. Nova Coluna: Porcentagem de Uso do Cartão
Inicialmente, é calculada a porcentagem de uso do cartão para cada usuário. O resultado é arredondado para duas casas decimais e armazenado na nova coluna `card_usage_percentage`. Isso proporciona uma visão clara da relação entre o uso do cartão e o limite disponível para cada usuário.

In [11]:
df['card_usage_percentage'] = round(df['card_usage']/df['card_limit'], 2)

### 3.2.2. Função: Generate Alert
Posteriormente, a função `generate_alert` é empregada para criar mensagens de alerta personalizadas. Essas mensagens variam conforme a porcentagem de uso do cartão, abrangendo diversos cenários de uso. Cada alerta oferece uma recomendação ou conselho pertinente, orientando o usuário com base na porcentagem de uso específica. Na função, espera-se na entrada uma tupla de 3 posições, com a porcentagem de uso do cartão, o primeiro nome do usuário e os últimos dígitos do cartão, respectivamente.

In [12]:
def generate_alert(user_inputs):
    
    usage_percentage = user_inputs[0]
    user_first_name = user_inputs[1]
    card_number_last_digits = user_inputs[2]
    
    if pd.isna(usage_percentage) is True:
        return f"🌟 Hey {user_first_name}, que tal aproveitar nossos incríveis cartões de crédito? Eles podem tornar suas compras ainda mais especiais!"
    
    alerts = [
        (1.0, f"⚠️ {user_first_name}, alerta importante! Seu cartão com final {card_number_last_digits} atingiu ou ultrapassou o limite. Hora de dar um respiro nos gastos! Avalie seus gastos recentes e crie um plano para evitar excessos futuros."),
        (0.9, f"🛑 Atenção, {user_first_name}! Você está quase no limite! Vamos diminuir o ritmo para manter tudo sob controle. Dica: Analise suas transações e identifique áreas onde pode cortar gastos desnecessários."),
        (0.8, f"📢 Ei, {user_first_name}, a gente percebeu que você está usando bastante seu cartão. Vamos dar um check nos gastos? Lembrete: Criar um orçamento pode ajudar a gerenciar suas despesas de forma eficiente."),
        (0.6, f"🔍 {user_first_name}, seus gastos estão moderados. Que tal dar uma olhada nas despesas para manter tudo nos eixos? Dica: Revisar suas transações pode ajudar a identificar oportunidades de economia."),
        (0.4, f"💡 {user_first_name}, ótimo equilíbrio nos gastos! Continue assim e sua conta vai sorrir de alegria! Sugestão: Considere criar um plano de economia para investir o dinheiro economizado."),
        (0.2, f"🚀 {user_first_name}, você está abaixo do limite! Mantenha o ritmo e continue controlando seus gastos com maestria! Dica: Explore opções de investimento para aumentar seus ganhos."),
        (0.0, f"🌈 {user_first_name}, seus gastos estão ótimos! Continue assim e seu cartão vai ficar muito feliz com você! Sugestão: Mantenha um registro das suas economias para acompanhar seu progresso.")
    ]
    
    for limit, message in alerts:
        if usage_percentage >= limit:
            return message
    
    return None

### 3.2.3. Função: Get Inputs
A função get_inputs extrai informações relevantes para a geração de alertas a partir do `DataFrame` com base no `user_id`. Esses dados incluem a porcentagem máxima de uso do cartão, o primeiro nome do usuário e os dígitos finais do número do cartão, que são os dados de entrada da função `generate_alert`.

In [13]:
def get_inputs(df, user_id):

    user_data = df[df['user_id'] == user_id]
    max_usage_pct = user_data['card_usage_percentage'].max()
    
    if pd.isna(max_usage_pct):
        user_id = user_data['user_id'].iloc[0]
        user_first_name = user_data['username'].iloc[0].split()[0]
        card_number_last_digits = user_data['card_number'].iloc[0]
    else:
        max_row = user_data[user_data['card_usage_percentage'] == max_usage_pct].iloc[0]
        user_id = max_row['user_id']
        user_first_name = max_row['username'].split()[0]
        card_number_last_digits = max_row['card_number'].split()[-1]
    
    return (max_usage_pct, user_first_name, card_number_last_digits)

### 3.2.4. Iteração e Preparação Para o Carregamento
Neste estágio final da transformação, ocorre a iteração pelos `user_id` únicos no `DataFrame`. Para cada `user_id`, um alerta é gerado usando a função `generate_alert`, e os dados necessários são obtidos por meio da função `get_inputs`. Esses dados são então agregados a uma lista chamada `data_to_load`, que resulta em pares de `user_id` e alertas personalizados. A lista `data_to_load` é o conjunto final de dados pronto para ser carregado no banco de dados.

In [14]:
data_to_load = []

for user_id in df['user_id'].unique():
    data_to_load.append((user_id, generate_alert(get_inputs(df, user_id))))

# 3.3. Carregamento

### 3.3.1. Instrução de Inserção
A instrução SQL `insert_news` é definida para inserir os dados de `user_id` e `mensagem` na tabela news.

In [15]:
insert_news = "INSERT INTO news(user_id, message) VALUES (?, ?)"

### 3.3.2. Carregando os Dados no Banco de Dados
Nesta etapa, os pares `user_id` e `message` são carregados na tabela `news`. Através da instrução `insert_news`, os dados da lista `data_to_load` são inseridos no banco de dados, assegurando que os alertas personalizados sejam associados aos respectivos usuários e, ainda, possam ser utilizados por outro processo do sistema para enviá-las aos usuários.

In [16]:
conn = sqlite3.connect('SDW2023.db')
cur = conn.cursor()

for data in data_to_load:
    cur.execute(insert_news, (int(data[0]), data[1]))

conn.commit()
cur.close()
conn.close()

### 3.3.3. Visualizando Resultados

Para finalizar o processo, visualizamos o carregamento através da execução de uma consulta SQL.

In [17]:
query = '''
    SELECT u.id AS user_id, u.name AS username, n.message as message
    FROM users u
    JOIN news n ON u.id = n.user_id
'''

conn = sqlite3.connect('SDW2023.db')
df = pd.read_sql(query, conn)
conn.close()

display(df)

,user_id,username,message
0,1,Maria Alice Santos,"🌟 Hey Maria, que tal aproveitar nossos incríve..."
1,2,Ronaldo Nunes,"⚠️ Ronaldo, alerta importante! Seu cartão com ..."
2,3,Paulo Moreira,"🌟 Hey Paulo, que tal aproveitar nossos incríve..."
3,4,Ana Rodrigues,"🔍 Ana, seus gastos estão moderados. Que tal da..."
4,5,Carlos Lima,"💡 Carlos, ótimo equilíbrio nos gastos! Continu..."
5,6,Lúcia Almeida,"⚠️ Lúcia, alerta importante! Seu cartão com fi..."
6,7,Fernando Sousa,"💡 Fernando, ótimo equilíbrio nos gastos! Conti..."
7,8,Mariana Costa,"⚠️ Mariana, alerta importante! Seu cartão com ..."
8,9,José Silva,"🚀 José, você está abaixo do limite! Mantenha o..."


# 5. Conclusão
A conclusão do desafio de ETL da <b>Santander Dev Week 2023</b> representa uma conquista sólida na jornada de manipulação de dados. Cada etapa, desde a extração até o carregamento nos bancos de dados, foi abordada com diligência e organização.

A transformação dos dados brutos resultou em alertas personalizados, fornecendo informações valiosas para os usuários tomarem decisões informadas sobre o uso de seus cartões de crédito.